## Training
Prepapre dataset with the prepare_dataset notebook, before running this one.

In [ ]:
from torch.utils.data import DataLoader
from tqdm import tqdm
from time import time, sleep
import numpy as np
import os
import warnings; warnings.filterwarnings("ignore")
from utils import *
  
# DEV = torch.device(CPU) # for debugging, use cpu
os.makedirs(f"{SAVE_DIR}", exist_ok=True)
print(f'DEV: {DEV}, has_screen: {LOCAL}, job id: {JOBID}')

# copy the python training to the directory (for cluster) (for local, it fails silently)
os.system(f"cp train.py {SAVE_DIR}/train.py")
os.system(f"cp utils.py {SAVE_DIR}/utils.py")

# SMALL, NORM, BIG = "small", "norm", "big"

# IDEAS

- 2 different opt1s for indepdendent nets, maybe 3 actually
- train only one branch at a time to prove it works
- add pre and post resclaing layers

In [ ]:

EPOCHS = 12 if LOCAL else 20 # number of epochs # 1000
# BATCH_SIZE = 128 # 128 
BATCH_SIZE = 4 if LOCAL else 64 # 64 <-

# LOAD_PRETRAINED = CURR_EVAL_MODEL if LOCAL else None # pretrained model
LOAD_PRETRAINED = None # Set it to None if you don't want to load pretrained model

In [ ]:
# schedulers
from numpy import concatenate as cat, linspace as linsp, logspace as logsp
### learning rate
# LR = 3e-4*logsp(0, -2, EPOCHS) 
# LR = 3e-3*np.ones(EPOCHS) 
# LR = 3e-3*logsp(0, -2, EPOCHS)  
LR = 1e-3*logsp(0, -2, EPOCHS) # <-
# LR = 3e-3*np.ones(EPOCHS) 

### loss ratios 
## gso
# RGSO = cat((linsp(1e-6, 3e-3, EPOCHS//2), linsp(3e-3, 0.0, EPOCHS//2))) 
MAX_GSO = 5e-4 # 3e-3 <-
# RGSO = cat((linsp(1e-6*MAX_GSO, MAX_GSO, EPOCHS//4), 
#             linsp(MAX_GSO, MAX_GSO, EPOCHS//4), 
#             linsp(MAX_GSO, 0.0, EPOCHS//4), 
#             linsp(0.0, 0.0, EPOCHS//4))) # <-
# RGSO = cat((MAX_GSO*logsp(-6, 0, EPOCHS//4), 
#                           MAX_GSO*logsp(0, 0, EPOCHS//4), 
#                           MAX_GSO*logsp(0, -10, EPOCHS//4), 
#                           logsp(-12, -12, EPOCHS//4))) 
RGSO = np.zeros(EPOCHS) 

## r1
MAX_R1 = 1.0 # 1.0 <-
R1 = MAX_R1 * np.ones(EPOCHS) 
# R1 = cat((np.ones(3*EPOCHS//4)*MAX_R1, 
#           np.zeros(EPOCHS//4))) # <-

## r2
MAX_R2 = 1.0
# R2 = MAX_R2 * np.ones(EPOCHS)
# R2 = cat((linsp(1e-6*MAX_R2, MAX_R2, EPOCHS//4),
#             linsp(MAX_R2, MAX_R2, EPOCHS//4), 
#             linsp(MAX_R2, 0.0, EPOCHS//4), 
#             linsp(0.0, 0.0, EPOCHS//4))) # <-
R2 = np.zeros(EPOCHS)

## r3
MAX_R3 = 1.0 # 1.0 <-
# R3 = MAX_R3 * np.ones(EPOCHS)
# R3 = cat((np.zeros(EPOCHS//4),
#          MAX_R3*np.ones(3*EPOCHS//4))) # <-
R3 = np.ones(EPOCHS) * MAX_R3

In [ ]:
#checks
if LOAD_PRETRAINED is not None: assert os.path.exists(LOAD_PRETRAINED), "Pretrained model does not exist"
assert os.path.exists(TRAIN_DS_PATH), "Training dataset does not exist"
assert os.path.exists(EVAL_DS_PATH), "Evaluation dataset does not exist"
assert os.path.exists(SAVE_DIR), "Save directory does not exist"
assert len(LR) == EPOCHS, "Learning rate array length does not match epochs"
assert len(RGSO) == EPOCHS, "GSO loss ratio array length does not match epochs"
assert len(R1) == EPOCHS, "R1 loss ratio array length does not match epochs"
assert len(R2) == EPOCHS, "R2 loss ratio array length does not match epochs"
assert len(R3) == EPOCHS, "R3 loss ratio array length does not match epochs"

In [ ]:
plt.figure(figsize=(12, 12))
plt.subplot(5, 1, 1)
plt.plot(LR, color='red')
plt.ylim(0, np.max(LR)*1.1)
plt.title('Learning Rate')
plt.subplot(5, 1, 2)
plt.plot(R1, color='red')
plt.ylim(0, np.max(R1)*1.1)
plt.title('R1 Loss Ratio')
plt.subplot(5, 1, 3)
plt.plot(R2, color='red')
plt.ylim(0, np.max(R2)*1.1)
plt.title('R2 Loss Ratio')
plt.subplot(5, 1, 4)
plt.plot(R3, color='red')
plt.ylim(0, np.max(R3)*1.1)
plt.title('R3 Loss Ratio')
plt.subplot(5, 1, 5)
plt.plot(RGSO, color='red')
plt.ylim(0, np.max(RGSO)*1.1)
plt.title('GSO Loss Ratio')
plt.show() if LOCAL else plt.savefig(f"{SAVE_DIR}/schedulers.png")

In [ ]:
# create training and evaluation datasets
train_ds = LiuqeDataset(TRAIN_DS_PATH)
val_ds = LiuqeDataset(EVAL_DS_PATH)
test_dataset()

In [ ]:
# test net I/O
test_network_io()

In [ ]:
# # ovveride networks for testing
# class LiuqeNet(Module):
#     def __init__(self, latent_size=32):
#         super(LiuqeNet, self).__init__()
#         assert latent_size % 2 == 0, "latent size should be even"
#         self.ngr, self.ngz = NGR, NGZ # grid size
#         #branch
#         self.branch = Sequential(
#             # View(-1, input_size),
#             Linear(NIN, 64), ActF(),
#             Linear(64, 32), ActF(),
#             Linear(32, latent_size), ActF(),
#         )
#         #trunk
#         def trunk_block(s): 
#             return  Sequential(
#                 # View(-1, s),
#                 Linear(s, 32), ActF(),
#                 Linear(32, latent_size//2), ActF(),
#             )
#         self.trunk_r, self.trunk_z = trunk_block(self.ngr), trunk_block(self.ngz)
#         # head
#         self.head = Sequential(
#             Linear(latent_size, 64), ActF(),
#             Linear(64, self.ngr*self.ngz), ActF(),
#             # View(-1, 1, *self.grid_size),
#         )
#     def forward(self, xb, r, z):
#         xb = self.branch(xb)
#         r, z = self.trunk_r(r), self.trunk_z(z) 
#         xt = torch.cat((r, z), 1) # concatenate
#         x = xt * xb # multiply trunk and branch
#         x = self.head(x) # head net
#         x = x.view(-1, 1, self.ngr, self.ngz) # reshape to grid
#         return x
    
# class LCFSNet(Module): 
#     def __init__(self):
#         super(LCFSNet, self).__init__()
#         self.lin = Sequential(
#             Linear(NIN, 64), ActF(),
#             Linear(64, 64), ActF(),
#             Linear(64, 2*NLCFS), ActF(),
#         )
#     def forward(self, x):
#         return self.lin(x) # linear layer


## Training

In [ ]:
def train():
    print("Ep:  || l1  | l2  | l3  | gso || lr  | r1  | r2  | r3  | gso || t | eta| improved    ")
    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True) # initialize DataLoader
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)  
    
    # network
    inet, gnet, h1, h2, h3 = InputNet(), GridNet(), FluxHead(), FluxHead(), LCFSHead() # initialize networks 
    model = LiuqeNet(inet, gnet, h1, h2, h3) # initialize model
    
    # net1 = LiuqeNet() # initialize model
    # net2 = LCFSNet() # initialize model


    
    opt1 = torch.optim.Adam(model.parameters(), lr=LR[0])
    opt1 = torch.optim.Adam( # Separate parameters for two optimizers
        list(gnet.parameters()) + list(h1.parameters()) + list(h2.parameters()),
        lr=LR[0]
    )
    opt2 = torch.optim.Adam(
        list(h3.parameters()) + list(inet.parameters()), 
        lr=LR[0]
    )

    # opt1 = torch.optim.Adam(net1.parameters(), lr=LR[0]) # optimizer for net1
    # opt2 = torch.optim.Adam(net2.parameters(), lr=LR[0]) # optimizer for net2

    if LOAD_PRETRAINED is not None: # load pretrained model
        model.load_state_dict(torch.load(LOAD_PRETRAINED, map_location=torch.device(CPU)), strict=STRICT_LOAD) # load pretrained model
        print(f"Pretrained model loaded: {LOAD_PRETRAINED}")

    model.to(DEV) # move model to DEV
    # net1.to(DEV) # move model to DEV
    # net2.to(DEV) # move model to DEV

    loss_fn = torch.nn.MSELoss() # Mean Squared Error Loss
    tlog, elog = np.inf*np.ones((EPOCHS, len(train_dl), len(LOSS_NAMES))), np.inf*np.ones((EPOCHS, len(val_dl), len(LOSS_NAMES))) # init log
    start_time = time() # start time
    for ep in range(EPOCHS): # epochs
        epoch_time = time()
        for pg in opt1.param_groups: pg['lr'] = LR[ep] # update learning rate
        for pg in opt2.param_groups: pg['lr'] = LR[ep] # update learning rate
        model.train()
        # net1.train() # training mode
        # net2.train() # training mode
        for ib, batch in enumerate(train_dl):
            if train_ds.on_dev: x, r, z, y1, y2, y3 = batch # unpack batch
            else: x, r, z, y1, y2, y3 = map(lambda t: t.to(DEV), batch) # move to DEV
            opt1.zero_grad(); opt2.zero_grad() # zero gradients
            yp1, yp2, yp3 = model(x, r, z) # forward pass
            # yp1 = net1(x, r, z) 
            # yp2 = y2
            # yp3 = net2(x) 
            gso, gsop = calc_gso_batch(y1, r, z, dev=DEV), calc_gso_batch(yp1, r, z, dev=DEV) # calculate grad shafranov
            # losses
            l1 = loss_fn(y1, yp1) 
            l2 = loss_fn(y2, yp2) 
            l3 = loss_fn(y3, yp3) 
            lgso = loss_fn(gsop, gso) # PINN loss on grad shafranov
            loss = R1[ep]*l1 + R2[ep]*l2 + R3[ep]*l3 + RGSO[ep]*lgso # total loss
            loss.backward() # backprop
            # l1.backward() # backprop
            # l3.backward() # backprop
            opt1.step(); opt2.step() # update weights
            tlog[ep, ib] = (l1.item(), l2.item(), l3.item(), lgso.item()) # save batch losses
        model.eval() # evaluation mode
        # net1.eval()
        # net2.eval()
        with torch.no_grad():
            for ib, batch in enumerate(val_dl):
                if val_ds.on_dev: x, r, z, y1, y2, y3 = batch # unpack batch
                else: x, r, z, y1, y2, y3 = map(lambda t: t.to(DEV), batch) # move to DEV
                yp1, yp2, yp3 = model(x, r, z) # forward pass
                # yp1 = net1(x, r, z)
                # yp2 = y2
                # yp3 = net2(x)
                gso, gsop = calc_gso_batch(y1, r, z, dev=DEV), calc_gso_batch(yp1, r, z, dev=DEV)
                # losses
                l1 = loss_fn(y1, yp1)
                l2 = loss_fn(y2, yp2)
                l3 = loss_fn(y3, yp3)
                lgso = loss_fn(gsop, gso)
                assert not torch.isnan(l1) and not torch.isnan(l2) and not torch.isnan(l3) and not torch.isnan(lgso), "Loss is NaN"
                elog[ep, ib] = (l1.item(), l2.item(), l3.item(), lgso.item()) # save batch losses
        # save model if improved        
        endp = "" 
        epoch_losses = np.mean(elog[ep,:,:], axis=0) # epoch losses: mean across batchess
        best_losses = np.min(np.mean(elog[:ep,:,:], axis=1), axis=0) if ep > 0 else epoch_losses # best losses: min across epochs
        for el, bl, n in zip(epoch_losses, best_losses, LOSS_NAMES):
            if el <= bl: torch.save(model.state_dict(), model_path(n)); endp+=f"*{n}"
            # if el <= bl: 
            #     torch.save(net1.state_dict(), f"{SAVE_DIR}/net1.pt")
            #     torch.save(net2.state_dict(), f"{SAVE_DIR}/net2.pt")
            #     endp+=f"*{n}"
        # print progress
        print(f"{ep+1:02d}/{EPOCHS:02d}||" +
            '|'.join([f"{v:.0e}" for v in epoch_losses]) + "||" +
            f"{LR[ep]:.0e}|{R1[ep]:.0e}|{R2[ep]:.0e}|{R3[ep]:.0e}|{RGSO[ep]:.0e}||" +
            f"{int(time()-epoch_time):02d}s|{int((time()-start_time)*(EPOCHS-ep)/(ep+1)/60):03d}m|", end=endp+'\n')
        if ep >= 10 and best_losses[2] > 0.1: return False #01/20|1e+00|1e+00|1e-01|1e+05||1e-03|1e+00|1e-18|1e+00|0e+00|00s|000m|*l1*l2*l3*gso stop training, if not converging, try again

    tlosses, elosses = np.mean(tlog, axis=1), np.mean(elog, axis=1) # losses across epochs
    print(f"Training time: {(time()-start_time)/60:.0f}mins")
    print("Best losses  : " + ", ".join([f"{n} {bl:.0e} (ep {be})" for n, bl, be in zip(LOSS_NAMES, np.min(elosses, axis=0), np.argmin(elosses, axis=0))]))
    print("Estimated MAE: " + ", ".join([f"{n} {np.sqrt(np.min(np.mean(elog[:,:,i], axis=1))):.3e}" for i, n in enumerate(LOSS_NAMES)]))
    np.save(f"{SAVE_DIR}/train_losses.npy", tlosses) # save training losses
    np.save(f"{SAVE_DIR}/eval_losses.npy", elosses) # save evaluation losses
    return True

# train the model (multiple attempts)
for i in range(20): 
    print('Starting training...')
    success = train()
    if success: break
    else: print(f"Convergence failed, retrying... {i+1}/20")
# if not success: delete the files and exit:
if not success: os.system(f"rm -rf {SAVE_DIR}")
assert success, "Training failed, no model saved"

In [ ]:
# plot losses
print("Plotting losses...")
fig, ax = plt.subplots(2, len(LOSS_NAMES), figsize=(4*len(LOSS_NAMES), 8))
# Load losses
train_loss = np.load(f"{SAVE_DIR}/train_losses.npy")
eval_loss = np.load(f"{SAVE_DIR}/eval_losses.npy")
for i, name in enumerate(LOSS_NAMES):
    # Linear scale
    ax[0,i].plot(train_loss[:,i], label='train')
    ax[0,i].plot(eval_loss[:,i], label='eval')
    ax[0,i].set_title(f"{name}")
    ax[0,i].set_xlabel("Epoch")
    ax[0,i].set_ylabel("Loss")
    ax[0,i].legend(); ax[0,i].grid(True)
    # Log scale
    ax[1,i].plot(train_loss[:,i], label='train')
    ax[1,i].plot(eval_loss[:,i], label='eval')
    ax[1,i].set_yscale('log')
    ax[1,i].set_title(f"{name} (log)")
    ax[1,i].set_xlabel("Epoch")
    ax[1,i].set_ylabel("Loss")
    ax[1,i].legend(); ax[1,i].grid(True)
plt.tight_layout()
plt.show() if LOCAL else plt.savefig(f"{SAVE_DIR}/losses.png")

In [ ]:
# testing network output
print("Testing network output...")
for ln in LOSS_NAMES:
    model = LiuqeNet(InputNet(), GridNet(), FluxHead(), FluxHead(), LCFSHead()) # initialize model
    model.load_state_dict(torch.load(model_path(ln), map_location=torch.device(CPU))) # load pretrained model
    plot_network_outputs(val_ds, model, title=ln) # plot network outputs

# net1 = LiuqeNet() # initialize model
# net2 = LCFSNet() # initialize model
# net1.load_state_dict(torch.load(f"{SAVE_DIR}/net1.pt", map_location=torch.device(CPU))) # load pretrained model
# net2.load_state_dict(torch.load(f"{SAVE_DIR}/net2.pt", map_location=torch.device(CPU))) # load pretrained model
# net1.eval() # evaluation mode
# net2.eval() # evaluation mode
# net1.to(CPU)
# net2.to(CPU)

# os.makedirs(f"{SAVE_DIR}/imgs", exist_ok=True)
# for i in np.random.randint(0, len(val_ds), 3 if LOCAL else 50):  
#     x, r, z, y1, _, y3 = val_ds[i] # get random sample
#     x, r, z, y1, y3 = map(lambda t: t.to(CPU), (x, r, z, y1, y3)) # move to DEV
#     x, r, z, y1, y3 = x.reshape(1,-1), r.reshape(1,NGR), z.reshape(1,NGZ), y1.reshape(1,1,NGZ,NGR), y3.reshape(1,2*NLCFS)
#     yp1 = net1(x, r, z)
#     yp3 = net2(x)
#     rr, zz = np.meshgrid(r.detach().cpu().numpy(), z.detach().cpu().numpy())
#     yp1 = yp1.detach().numpy().reshape(NGZ,NGR)
#     y1 = y1.detach().numpy().reshape(NGZ,NGR)
#     yp3 = yp3.detach().numpy().reshape(2*NLCFS)
#     y3 = y3.detach().numpy().reshape(2*NLCFS)
#     s1 = 4
#     plt.figure(figsize=(10, 5))
#     plt.subplot(1, 3, 1)
#     plt.scatter(rr,zz, c=y1, s=s1)
#     # plt.plot(y3[:NLCFS], y3[NLCFS:], lw=2)
#     plt.colorbar()
#     plot_vessel()
#     plt.axis('equal')
#     plt.subplot(1, 3, 2)
#     plt.scatter(rr,zz, c=yp1, s=s1)
#     # plt.plot(yp3[:NLCFS], yp3[NLCFS:], lw=2)
#     plt.colorbar()
#     plot_vessel()
#     plt.axis('equal')
#     plt.subplot(1, 3, 3)
#     plt.scatter(rr,zz, c=np.abs(y1-yp1), s=s1)
#     plt.colorbar()
#     plot_vessel()
#     plt.axis('equal')
#     plt.suptitle(f"Sample {i} (y1)")
#     plt.tight_layout()
#     plt.show() if LOCAL else plt.savefig(f"{SAVE_DIR}/imgs/net_example_l1_{i}.png")
#     plt.close()
    

In [ ]:
# test LCFS net
m = LiuqeNet(InputNet(), GridNet(), FluxHead(), FluxHead(), LCFSHead()) # initialize model
m.load_state_dict(torch.load(model_path('l3'), map_location=torch.device(CPU))) # load pretrained model
lcfs = LCFSNet(m.input_net, m.lcfs_head)

# lcfs = LCFSNet() # initialize model
# lcfs.load_state_dict(torch.load(f"{SAVE_DIR}/net2.pt", map_location=torch.device(CPU))) # load pretrained model
plot_lcfs_net_out(val_ds, lcfs, title='') # plot LCFS net outputs

In [ ]:
# test inference speed
print("Testing inference speed...")
try:
    model = LiuqeNet(InputNet(), GridNet(), FluxHead(), FluxHead(), LCFSHead()) # initialize model
    model.load_state_dict(torch.load(model_path(LOSS_NAMES[2]), map_location=torch.device(CPU))) # load pretrained model
    model.eval()
    ds = val_ds
    n_samples = 100
    random_idxs = np.random.choice(n_samples, len(ds))
    #cpu
    cpu_times1, cpu_times2 = [], []
    for i in random_idxs:
        start_t = time()
        x, r, z, y1, y2, y3 = ds[i]
        x, r, z = x.to(CPU), r.to(CPU), z.to(CPU)
        x = x.view(1, -1)
        r = r.view(1, NGZ)
        z = z.view(1, NGR)
        start_t2 = time()
        yp = model(x, r, z)
        end_t = time()
        cpu_times1.append(end_t - start_t)
        cpu_times2.append(end_t - start_t2)
    # DEV
    model.to(DEV)
    dev_times1, dev_times2 = [], []
    for i in random_idxs:
        x, r, z, y1, y2, y3 = ds[i]
        x, r, z = x.to(DEV), r.to(DEV), z.to(DEV)
        x = x.view(1, -1)
        r = r.view(1, NGZ)
        z = z.view(1, NGR)
        start_t = time()
        start_t2 = time()
        yp = model(x, r, z)
        end_t = time()
        dev_times1.append(end_t - start_t)
        dev_times2.append(end_t - start_t2)
    cpu_times1, dev_times1 = np.array(cpu_times1)*1000, np.array(dev_times1)*1000
    cpu_times2, dev_times2 = np.array(cpu_times2)*1000, np.array(dev_times2)*1000
    print(f"cpu: inference time: [full -> {cpu_times1.mean():.5f}ms, std: {cpu_times1.std():.5f}]")
    print(f"cpu: inference time: [inference only -> {cpu_times2.mean():.5f}ms, std: {cpu_times2.std():.5f}]")
    print(f"dev: inference time: [full -> {dev_times1.mean():.5f}ms, std: {dev_times1.std():.5f}]")
    print(f"dev: inference time: [inference only -> {dev_times2.mean():.5f}ms, std: {dev_times2.std():.5f}]")
except Exception as e:
    print(f"Error in inference speed test: {e}")
    print("Inference speed test failed")
    pass

In [ ]:
# convert best l3 network to ONNX
try:
    print("Converting best l3 network to ONNX...")
    m = LiuqeNet(InputNet(), GridNet(), FluxHead(), FluxHead(), LCFSHead()) # initialize model
    m.load_state_dict(torch.load(model_path(LOSS_NAMES[2]), map_location=torch.device(CPU))) # load pretrained model
    net = LCFSNet(m.input_net, m.lcfs_head)
    net.to(CPU)  

    dummy_input = torch.randn(1, NIN, device=CPU)
    try:
        torch.onnx.export(net, dummy_input, f'{SAVE_DIR}/net.onnx', export_params=True,
                        opset_version=12, do_constant_folding=True,
                        input_names=['x'], output_names=['y'])
        print("ONNX model saved")
    except Exception as e: print(f"Error exporting to ONNX: {e}")
except Exception as e:
    print(f"Error in ONNX conversion: {e}")
    print("ONNX conversion failed")
    pass

In [ ]:
print(f"{JOBID} done")
if not LOCAL: sleep(30) # wait for files to update (for cluster)
#copy the log file to the folder
os.system(f"cp jobs/{JOBID}.txt {SAVE_DIR}/log.txt")